In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
#nltk.download()

In [2]:
df_hashtag = pd.read_csv('../opColombiaResiste.csv')
df_anon_users = pd.read_csv('../tweetsopColombiaResiste.csv')
df = pd.concat([df_hashtag, df_anon_users],sort=False)

In [3]:
df.head()

,user,id_user,timestamp,text,id_tweet,Reply_to,Coordinates,retweets_count,Name,Location,...,Favourites_Count,User_Mentions,User_Ids_Mentions,Favorited,Reply_To_UserName,Num_Friends,Listed_count,link_tweet,replies_count,likes_count
0,@Anons_MexCity,1220555617313492995,Sun Feb 09 13:48:23 +0000 2020,RT @ActualidadRT: Israel ya diseña nuevos mapa...,1226503109242953733,NaN,NaN,37,Anonymous Ciudad de Mexico,ciudad de mexico,...,129.0,@ActualidadRT,100731315,False,NaN,121.0,0.0,twitter.com/@Anons_MexCity/status/122650310924...,NaN,NaN
1,@Anons_MexCity,1220555617313492995,Sun Feb 09 13:47:26 +0000 2020,RT @El_Universal_Mx: El fármaco ha logrado cur...,1226502871220391936,NaN,NaN,36,Anonymous Ciudad de Mexico,ciudad de mexico,...,129.0,@El_Universal_Mx,16676396,False,NaN,121.0,0.0,twitter.com/@Anons_MexCity/status/122650287122...,NaN,NaN
2,@Anons_MexCity,1220555617313492995,Sun Feb 09 13:46:29 +0000 2020,RT @El_Universal_Mx: Sólo 392 de 26 mil 700 po...,1226502632887418881,NaN,NaN,9,Anonymous Ciudad de Mexico,ciudad de mexico,...,129.0,@El_Universal_Mx,16676396,False,NaN,121.0,0.0,twitter.com/@Anons_MexCity/status/122650263288...,NaN,NaN
3,@Anons_MexCity,1220555617313492995,Sun Feb 09 13:45:34 +0000 2020,RT @vanguardiamx: https://t.co/7hAMnMMXA5,1226502402397872130,NaN,NaN,1,Anonymous Ciudad de Mexico,ciudad de mexico,...,129.0,@vanguardiamx,18193312,False,NaN,121.0,0.0,twitter.com/@Anons_MexCity/status/122650240239...,NaN,NaN
4,@Anons_MexCity,1220555617313492995,Sun Feb 09 13:45:00 +0000 2020,RT @Hispantv: #Mexico continúa recibiendo a lo...,1226502259468636160,NaN,NaN,7,Anonymous Ciudad de Mexico,ciudad de mexico,...,129.0,@Hispantv,213589457,False,NaN,121.0,0.0,twitter.com/@Anons_MexCity/status/122650225946...,NaN,NaN


In [4]:
def clean_text(x):
    tokens = x.split(" ")
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words


In [5]:
df['clean_text'] = df['text'].map(lambda x: clean_text(x))

In [6]:
def get_hashtags_operations(x):
    tokens = x.split(" ")
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove break lines
    tokens = [w.replace('\n','') for w in tokens]
    # get hashtags
    hashtags = [token for token in tokens if '#' in token]
    # get operation hashtags
    hashtags = [hashtag for hashtag in hashtags if '#op' == hashtag[:3]]
    return hashtags

In [7]:
df['hashtags'] = df['text'].map(lambda x: get_hashtags_operations(x))

In [8]:
df['user'].value_counts()

@AnonOpsColombia    219
@AnonymousIbero     210
@AnonimoCol_H       204
@AnonymousNewz      196
@Anons_MexCity      195
@IberoAnonOps       194
@AnonPiaui           85
@ahhappe              3
@HorizonteIndep1      2
@Infinity__Corp       2
@hmsjulio             2
@johnnybrav9          1
@HarrisonCaSa         1
@AAguacatera          1
@RobertoMTico         1
@HSAV18               1
@federicxo79          1
@OpChileDesperto      1
@Annonnymous_R        1
@DanRodBoh            1
@Tabuto_              1
@Krephy               1
@MaryCarvajal15       1
@BitoCasCub           1
Name: user, dtype: int64

In [9]:
terms_attacks = ['tango','down','ddos','attack','pastebin','hacked','defaced','deface','tangodown','zone-h','hastebin','leaked','site','espelho','downed','checkhost','hackeado','target']

In [10]:
df['attack'] = df['clean_text'].map(lambda x: True if len([term for term in terms_attacks if term in x])>0 else False)
df['operations'] = df['hashtags'].map(lambda x: True if len([hashtag for hashtag in x if '#op' == hashtag[:3]])>0 else False)
df['RT'] = df['clean_text'].map(lambda x: True if 'rt' in x else False)

In [11]:
df[df['attack']==True].head()

,user,id_user,timestamp,text,id_tweet,Reply_to,Coordinates,retweets_count,Name,Location,...,Num_Friends,Listed_count,link_tweet,replies_count,likes_count,clean_text,hashtags,attack,operations,RT
148,@Anons_MexCity,1220555617313492995,Sun Jan 26 00:42:32 +0000 2020,A todos los invitamos a nuestro ataque DDOS ba...,1221231916000497667,NaN,NaN,1,Anonymous Ciudad de Mexico,ciudad de mexico,...,121.0,0.0,twitter.com/@Anons_MexCity/status/122123191600...,NaN,NaN,"[todos, los, invitamos, nuestro, ataque, ddos,...",[#opdomesticterrorism!],True,True,False
205,@AnonymousNewz,1327529652,Wed Jan 08 14:57:01 +0000 2020,RT @realTuckFrumper: Hackers Deface U.S. Gover...,1214923969985490947,NaN,NaN,10,Anonymous,Justo detras de ti,...,98.0,19.0,twitter.com/@AnonymousNewz/status/121492396998...,NaN,NaN,"[rt, realtuckfrumper, hackers, deface, us, gov...",[],True,False,True
278,@AnonymousNewz,1327529652,Sun Dec 01 02:46:54 +0000 2019,RT @S0u1_HLoTW: https://t.co/vXcBiNngqK tango ...,1200969490256805888,NaN,NaN,36,Anonymous,Justo detras de ti,...,98.0,19.0,twitter.com/@AnonymousNewz/status/120096949025...,NaN,NaN,"[rt, httpstcovxcbinngqk, tango, youranonriots]",[],True,False,True
422,@IberoAnonOps,1496684839,Fri Nov 15 23:32:25 +0000 2019,#OpChileDesperto TANGO DOWN\nhttps://t.co/2joE...,1195484729564389376,NaN,NaN,0,Operaciones de Anonymous,Justo detras de ti,...,71.0,21.0,twitter.com/@IberoAnonOps/status/1195484729564...,NaN,NaN,"[opchiledesperto, tango, httpstconfanjtspfg]",[#opchiledesperto],True,True,False
444,@IberoAnonOps,1496684839,Mon Aug 22 15:55:25 +0000 2016,RT @AnonymousIbero: #TangoDown @Ecatepec el p...,767752040491601920,NaN,NaN,5,Operaciones de Anonymous,Justo detras de ti,...,71.0,21.0,twitter.com/@IberoAnonOps/status/7677520404916...,NaN,NaN,"[rt, anonymousibero, tangodown, ecatepec, el, ...",[],True,False,True


In [12]:
df[df['operations']==True].head()

,user,id_user,timestamp,text,id_tweet,Reply_to,Coordinates,retweets_count,Name,Location,...,Num_Friends,Listed_count,link_tweet,replies_count,likes_count,clean_text,hashtags,attack,operations,RT
122,@Anons_MexCity,1220555617313492995,Tue Jan 28 14:24:04 +0000 2020,RT @AnonymousVideo: ►WE WILL FIGHT AND DESTROY...,1222163437322522626,NaN,NaN,33,Anonymous Ciudad de Mexico,ciudad de mexico,...,121.0,0.0,twitter.com/@Anons_MexCity/status/122216343732...,NaN,NaN,"[rt, anonymousvideo, fight, destroy, neo, nazi...","[#opdomesticterrorism, #opkkk, #opaltright, #o...",False,True,True
148,@Anons_MexCity,1220555617313492995,Sun Jan 26 00:42:32 +0000 2020,A todos los invitamos a nuestro ataque DDOS ba...,1221231916000497667,NaN,NaN,1,Anonymous Ciudad de Mexico,ciudad de mexico,...,121.0,0.0,twitter.com/@Anons_MexCity/status/122123191600...,NaN,NaN,"[todos, los, invitamos, nuestro, ataque, ddos,...",[#opdomesticterrorism!],True,True,False
149,@Anons_MexCity,1220555617313492995,Sat Jan 25 22:21:14 +0000 2020,#AnonymousCiudadDeMexico Nuestro mensaje a tod...,1221196354736668672,NaN,NaN,11,Anonymous Ciudad de Mexico,ciudad de mexico,...,121.0,0.0,twitter.com/@Anons_MexCity/status/122119635473...,NaN,NaN,"[anonymousciudaddemexico, nuestro, mensaje, to...","[#opmexico, #opchiledesperto, #opboliviafuerte]",False,True,False
193,@AnonymousNewz,1327529652,Wed Feb 05 14:10:00 +0000 2020,#OpChileDesperto https://t.co/DiuUbfuXCr,1225058999948709894,NaN,NaN,1,Anonymous,Justo detras de ti,...,98.0,19.0,twitter.com/@AnonymousNewz/status/122505899994...,NaN,NaN,"[opchiledesperto, httpstcodiuubfuxcr]",[#opchiledesperto],False,True,False
194,@AnonymousNewz,1327529652,Wed Feb 05 01:51:41 +0000 2020,#OpChileDesperto Anonymous hackea y filtra dat...,1224873195951599616,NaN,NaN,58,Anonymous,Justo detras de ti,...,98.0,19.0,twitter.com/@AnonymousNewz/status/122487319595...,NaN,NaN,"[opchiledesperto, anonymous, hackea, filtra, d...",[#opchiledesperto],False,True,False


In [13]:
df_top_hashtags = df[df['operations']==True]['hashtags'].explode().value_counts().to_frame()
hashtags = df_top_hashtags[df_top_hashtags['hashtags']>3].to_dict()['hashtags']

In [14]:
dict_hashtags_rel = {}
for hashtag in hashtags.keys():
    df_hashtag = df[df['operations']==True][['hashtags','user']]
    df_hashtag[hashtag] = df_hashtag['hashtags'].apply(lambda x : True if hashtag in x else False)
    dict_hashtags_rel[hashtag] = list(df_hashtag.loc[df_hashtag[hashtag],'user'].value_counts().to_frame().index.values)

In [15]:
dict_output = {}
list_users = []
for index, row in df[(df['attack']==True) & (df['RT']==False)].iterrows():
    if row['user'] in list_users:
        dict_output[row['user']].append({"id":row['link_tweet'],"name":"Attack","value":row['retweets_count'],"type":"#931621"})
    else:
        list_users.append(row['user'])
        dict_output[row['user']]=[]
        dict_output[row['user']].append({"id":row['link_tweet'],"name":"Attack","value":row['retweets_count'],"type":"#931621"})

In [16]:
dict_output_formatted = []
for elem in dict_output:
    dict_output_formatted.append({"id":elem,"name":elem,"value":"","collapsed":"true","children":dict_output[elem],'type':'#2e0e02'})
for elem in hashtags.keys():
    if len(list(set(dict_output)&set(dict_hashtags_rel[elem])))>0:
        dict_output_formatted.append({"id":elem,"name":elem,"value":hashtags[elem],"collapsed":"true","linkWith":dict_hashtags_rel[elem],'type':'#ce6c47'})

In [17]:
dict_output_formatted

[{'id': '@Anons_MexCity',
  'name': '@Anons_MexCity',
  'value': '',
  'collapsed': 'true',
  'children': [{'id': 'twitter.com/@Anons_MexCity/status/1221231916000497667',
    'name': 'Attack',
    'value': 1,
    'type': '#931621'}],
  'type': '#44ffd2'},
 {'id': '@IberoAnonOps',
  'name': '@IberoAnonOps',
  'value': '',
  'collapsed': 'true',
  'children': [{'id': 'twitter.com/@IberoAnonOps/status/1195484729564389376',
    'name': 'Attack',
    'value': 0,
    'type': '#931621'}],
  'type': '#44ffd2'},
 {'id': '@AnonymousIbero',
  'name': '@AnonymousIbero',
  'value': '',
  'collapsed': 'true',
  'children': [{'id': 'twitter.com/@AnonymousIbero/status/1203495092172054530',
    'name': 'Attack',
    'value': 1,
    'type': '#931621'},
   {'id': 'twitter.com/@AnonymousIbero/status/1202271644821213190',
    'name': 'Attack',
    'value': 1,
    'type': '#931621'}],
  'type': '#44ffd2'},
 {'id': '@AnonimoCol_H',
  'name': '@AnonimoCol_H',
  'value': '',
  'collapsed': 'true',
  'children'